## Download specific versions and source code locally

In [1]:
# torch==1.10.2
!pip install transformers==4.16.2 datasets==1.17.0 tokenizers==0.11.6 wandb==0.12.14

     |████████████████████████████████| 3.5 MB 5.5 MB/s 
     |████████████████████████████████| 306 kB 52.5 MB/s 
     |████████████████████████████████| 6.5 MB 34.9 MB/s 
     |████████████████████████████████| 1.8 MB 36.0 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 37.4 MB/s 
     |████████████████████████████████| 895 kB 36.8 MB/s 
     |████████████████████████████████| 136 kB 51.8 MB/s 
     |████████████████████████████████| 212 kB 50.8 MB/s 
     |████████████████████████████████| 1.1 MB 45.1 MB/s 
     |████████████████████████████████| 144 kB 45.0 MB/s 
     |████████████████████████████████| 181 kB 51.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 144 kB 51.3 MB/s 
     |████████████████████████████████| 271 kB 45.9 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8

In [1]:
!nvidia-smi

Mon Apr 11 15:54:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    76W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%cd /content
!rm -r ntu_nlp_al
!git clone -b kretov/adding_tagging https://github.com/DevKretov/ntu_nlp_al

/content
Cloning into 'ntu_nlp_al'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 131 (delta 68), reused 99 (delta 41), pack-reused 0
Receiving objects: 100% (131/131), 56.54 MiB | 11.72 MiB/s, done.
Resolving deltas: 100% (68/68), done.
Checking out files: 100% (24/24), done.


In [4]:

%cd ntu_nlp_al/

/content/ntu_nlp_al


## Initialisation

In [5]:
import torch
from transformers import get_scheduler
from datasets import list_metrics, load_metric

from active_learning_trainer import ALTrainer
from transformers import AutoTokenizer
from dataset import Dataset, TokenClassificationDataset
from model import Model


import datetime
import wandb


2022-04-11 15:54:40,780 git.cmd DEBUG:Popen(['git', 'version'], cwd=/content/ntu_nlp_al, universal_newlines=False, shell=None, istream=None)
2022-04-11 15:54:40,808 git.cmd DEBUG:Popen(['git', 'version'], cwd=/content/ntu_nlp_al, universal_newlines=False, shell=None, istream=None)
2022-04-11 15:54:41,174 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-11 15:54:41,331 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /bert-base-multilingual-uncased/resolve/main/config.json HTTP/1.1" 200 0


In [6]:
parameters = dict()
parameters['use_gpu'] = True

parameters['weights_and_biases_on'] = True
parameters['weights_and_biases_key'] = '5e5e00356042a33b5cb271399b8d05c9c9d6ded8'
# TODO: run name based on timestamp
current_timestamp = str(datetime.datetime.now()).split('.')[0]

# TODO: implement it
parameters['weights_and_biases_save_predictions'] = False

parameters['pretrained_model_name'] = 'prajjwal1/bert-small' #'distilbert-base-uncased'


# parameters['train_dataset_file_path'] = 'data/imdb/train_IMDB.csv'
# parameters['val_dataset_file_path'] = 'data/imdb/test_IMDB.csv'
# parameters['test_dataset_file_path'] = 'data/imdb/test_IMDB.csv'

parameters['dataset_from_datasets_hub'] = False
parameters['dataset_from_datasets_hub_name'] = 'conll2003'
parameters['train_dataset_file_path'] = 'data/news/train.csv'
parameters['val_dataset_file_path'] = 'data/news/val.csv'
parameters['test_dataset_file_path'] = 'data/news/test.csv'
parameters['dataset_file_delimiter'] = ','

### FIRST IS FOR TAGGING
parameters['dataset_text_column_name'] =  'tokens' #'text'
parameters['dataset_label_column_name'] = 'ner_tags'#'airline_sentiment'

### THIS IS FOR CLASSIFICATION
### FIRST OF ALL, LOOK AT COLUMNS IN DATASET CSV!!! THEN WRITE HERE!
parameters['dataset_text_column_name'] = 'text_cleaned'  # 'text'
parameters['dataset_label_column_name'] = 'label_reduced'  # 'airline_sentiment'

# TODO: implement this with CrossEntropyLoss
parameters['loss'] = 'cross_entropy'
parameters['loss_weighted'] = False

parameters['class_imbalance_reweight'] = True
parameters['train_batch_size'] = 32
parameters['val_batch_size'] = 64
parameters['test_batch_size'] = 64
parameters['epochs'] = 5
# parameters['finetuned_model_type'] = 'classification'
parameters['finetuned_model_type'] = 'classification'
model_type = parameters['finetuned_model_type']


parameters['al_iterations'] = 100
parameters['init_dataset_size'] = 32
parameters['add_dataset_size'] = 32
### HERE CAN BE least_confidence OR badge OR random OR kmeans OR entropy
parameters['al_strategy'] = 'kmeans' #'least_confidence'
# IF YOU WANT TO TRAIN ON THE WHOLE MODEL
parameters['full_train'] = False

parameters['debug'] = False

In [7]:
device = 'cpu'
if parameters['use_gpu']:
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

parameters['weights_and_biases_run_name'] = f'{current_timestamp}_run_{model_type}_{device}'
print(f'Device set to {device}!')

Device set to cuda:0!


## Preparing tokenizer, dataset, model

In [8]:
tokenizer = AutoTokenizer.from_pretrained(parameters['pretrained_model_name'])

dataset_obj = None
if parameters['finetuned_model_type'] == 'classification':
    dataset_obj = Dataset(tokenizer)
elif parameters['finetuned_model_type'] == 'tagging':
    dataset_obj = TokenClassificationDataset(tokenizer)
else:
    model_type = parameters['finetuned_model_type']
    raise NotImplementedError(f'Type {model_type} not supported yet!')

2022-04-11 15:54:41,652 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-11 15:54:41,787 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
2022-04-11 15:54:41,795 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-11 15:54:41,936 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/config.json HTTP/1.1" 200 0
2022-04-11 15:54:41,981 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-11 15:54:42,114 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
2022-04-11 15:54:42,122 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-11 15:54:42,256 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-sma

In [9]:
if parameters['dataset_from_datasets_hub']:
    dataset_name = parameters['dataset_from_datasets_hub_name']
    dataset_obj.load_hosted_dataset(dataset_name)
else:
    data_files = {
        'train': [parameters['train_dataset_file_path']],
        'val': [parameters['val_dataset_file_path']],
        'test': [parameters['test_dataset_file_path']]
    }

    dataset_obj.load_csv_dataset(
        data_files,
        delimiter=parameters['dataset_file_delimiter']
    )

dataset_obj.truncate_dataset('train', 10000)
dataset_obj.truncate_dataset('val', 1000)
dataset_obj.truncate_dataset('test', 10000)

dataset_obj.prepare_dataset(parameters['dataset_label_column_name'], parameters['dataset_text_column_name'], )
# dataset_obj.prepare_labels(parameters['dataset_label_column_name'])
# dataset_obj.encode_dataset(parameters['dataset_text_column_name'])

print(f'Categories: {dataset_obj.get_all_categories()}')
num_labels = dataset_obj.get_num_categories()

2022-04-11 15:54:43,297 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): s3.amazonaws.com:443
2022-04-11 15:54:43,510 urllib3.connectionpool DEBUG:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/csv/csv.py HTTP/1.1" 200 0
2022-04-11 15:54:43,521 datasets.builder WARNING:Using custom data configuration default-282ea53093e4eac7


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-282ea53093e4eac7/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/2808 [00:00<?, ?ex/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Categories: {'alt': 0, 'comp': 1, 'misc': 2, 'rec': 3, 'sci': 4, 'soc': 5, 'talk': 6}


In [10]:
model = Model(
        parameters['pretrained_model_name'],
        model_type=parameters['finetuned_model_type'],
        num_labels=num_labels
    )

2022-04-11 15:55:00,184 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-11 15:55:00,322 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/config.json HTTP/1.1" 200 0
2022-04-11 15:55:00,332 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): huggingface.co:443
2022-04-11 15:55:00,465 urllib3.connectionpool DEBUG:https://huggingface.co:443 "HEAD /prajjwal1/bert-small/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initi

### Connect to W&B environment

In [11]:
if parameters['weights_and_biases_on']:
    wandb.login(key='5e5e00356042a33b5cb271399b8d05c9c9d6ded8')
    wandb.init(
        name=parameters['weights_and_biases_run_name'],
        project='ntu_al',
        reinit=True
    )

    wandb.config.update(parameters)
    wandb.watch(model.model)

2022-04-11 15:55:06,161 urllib3.connectionpool DEBUG:Starting new HTTP connection (1): 172.28.0.2:9000
2022-04-11 15:55:06,169 urllib3.connectionpool DEBUG:http://172.28.0.2:9000 "GET /api/sessions?token= HTTP/1.1" 200 420
2022-04-11 15:55:06,187 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): api.wandb.ai:443
2022-04-11 15:55:06,307 urllib3.connectionpool DEBUG:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
2022-04-11 15:55:06,316 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): api.wandb.ai:443
2022-04-11 15:55:06,372 urllib3.connectionpool DEBUG:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
wandb: Currently logged in as: kretov (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai t

### Init trainer and combine everything

In [12]:
trainer = ALTrainer(
    wandb_on=parameters['weights_and_biases_on'],
    imbalanced_training=parameters['class_imbalance_reweight'],
    model_type=parameters['finetuned_model_type']
)
trainer.set_model(model)

# TODO: add strategy
trainer.set_strategy(None)
trainer.set_dataset(dataset_obj)
dataset_obj.prepare_dataloaders(
    train_batch_size=parameters['train_batch_size'],
    val_batch_size=parameters['val_batch_size'],
    test_batch_size=parameters['test_batch_size'],
)

optimizer = torch.optim.AdamW(model.model.parameters(), lr=5e-5)
trainer.set_optimizer(optimizer)

num_training_steps = parameters['epochs'] * trainer.get_training_steps_num()

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
trainer.set_lr_scheduler(lr_scheduler)
trainer.set_device(device)

trainer.add_evaluation_metric(load_metric('accuracy'))

if parameters['finetuned_model_type'] == 'tagging':
    trainer.add_evaluation_metric(load_metric('seqeval'))
else:
    trainer.add_evaluation_metric(load_metric('f1'))
    trainer.add_evaluation_metric(load_metric('precision'))
    trainer.add_evaluation_metric(load_metric('recall'))

2022-04-11 15:55:15,478 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): s3.amazonaws.com:443
2022-04-11 15:55:15,643 urllib3.connectionpool DEBUG:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/accuracy/accuracy.py HTTP/1.1" 200 0
2022-04-11 15:55:15,650 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): raw.githubusercontent.com:443
2022-04-11 15:55:15,698 urllib3.connectionpool DEBUG:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.17.0/metrics/accuracy/accuracy.py HTTP/1.1" 200 0
2022-04-11 15:55:15,720 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): s3.amazonaws.com:443
2022-04-11 15:55:15,865 urllib3.connectionpool DEBUG:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/f1/f1.py HTTP/1.1" 200 0
2022-04-11 15:55:15,870 urllib3.connectionpool DEBUG:Starting new HTTPS connection (1): raw.githubusercontent.com:443
2022-04-11 15:55:15,916 urllib3.connectionpool DEBUG:

## Train

In [ ]:
if parameters['full_train']:
    trainer.full_train(
        train_epochs=parameters['epochs'],
        train_batch_size=parameters['train_batch_size'],
        val_batch_size=parameters['val_batch_size'],
        test_batch_size=parameters['test_batch_size'],
        debug=parameters['debug']
    )

trainer.al_train(
    al_iterations=parameters['al_iterations'],
    init_dataset_size=parameters['init_dataset_size'],
    add_dataset_size=parameters['add_dataset_size'],
    train_epochs=parameters['epochs'],
    strategy=parameters['al_strategy'],
    train_batch_size=parameters['train_batch_size'],
    val_batch_size=parameters['val_batch_size'],
    test_batch_size=parameters['test_batch_size'],
    debug=parameters['debug']
)

2022-04-11 15:55:16,439 root INFO:Training initialized!


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/32 [00:00<?, ?ex/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/9968 [00:00<?, ?ex/s]

...

Training mean loss: 1.4324580033620198: 100%|██████████| 3/3 [00:02<00:00,  1.47it/s]

2022-04-11 16:00:50,546 root INFO:Epoch finished. Evaluation:
2022-04-11 16:00:50,554 root INFO:

Epoch   5


Training mean loss: 1.267296592394511: 100%|██████████| 3/3 [00:01<00:00,  1.52it/s]

2022-04-11 16:00:52,545 root INFO:Epoch finished. Evaluation:
2022-04-11 16:00:52,548 root DEBUG:Is model on CUDA - cuda:0
2022-04-11 16:00:52,550 root INFO:Running Eval mode...


  0%|          | 0/16 [00:00<?, ?it/s]